Convolutional Neural Network
https://realpython.com/python-keras-text-classification/

In [1]:
import sqlite3
import pandas as pd

df = pd.read_csv('preprocessed_data.csv', index_col=0)

In [2]:
from sklearn.model_selection import train_test_split

functions = df['function'].values
y = df['isVulnerable'].values

functions_train, functions_test, y_train, y_test = train_test_split(functions, y, test_size=0.2, random_state=42)


In [3]:
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import wordpunct_tokenize

tokenizer = Tokenizer(analyzer=wordpunct_tokenize)
tokenizer.fit_on_texts(functions_train)

X_train = tokenizer.texts_to_sequences(functions_train)
X_test = tokenizer.texts_to_sequences(functions_test)

vocab_size = len(tokenizer.word_index) + 1


2023-04-30 13:00:06.841169: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 13:00:06.878944: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 13:00:06.879388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 13:00:07.491103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
print(vocab_size)

54797


In [5]:
from keras.utils import pad_sequences

maxlen = 1000

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0])

[   41 37081     1    10    91    12    91     4   122    12  1966     7
     9    10  7691    12   538     6  1731    91     2  5138     5   382
    14   538     2   101     5     8     1   538     2 17660     7     9
   538     2 37082     6    15     3    86     1   538     2 17660     5
   538     2 17660     6    35     3    11     8     1  2359     1   538
     4 13530    28     9   555    14   538     2 23429     5  4799     1
   538     4 13530     5    11 37083    14    91     2  8529     5     8
     1   538     2 13882     7     9     8   108  2359     1   538     4
 25100     7    38   115    28     9     8     1  2359     1   538     4
  5431     7    38   118     7     9 25101     1   538     4  3686     5
     8     1 13531     1   538     7    38  4094     7     9  6394    14
   538     2  4800     4 13532     5    11    48     9   122    19     6
  4094     3    10 15441    12 25102     6  1731    91     2 25103    13
 25104     5   122    12 25105     6    91     2 25

In [7]:
from keras.models import Sequential
from keras import layers

embedding_dim=100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 100)         5479700   
                                                                 
 conv1d (Conv1D)             (None, 996, 128)          64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,545,129
Trainable params: 5,545,129
Non-trainable params: 0
____________________________________________

In [9]:
from keras.backend import clear_session
clear_session()

history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
1960/1960 [==============================] - 169s 86ms/step - loss: 0.6672 - accuracy: 0.5446 - val_loss: 0.6307 - val_accuracy: 0.5905
Epoch 2/10
1960/1960 [==============================] - 159s 81ms/step - loss: 0.5840 - accuracy: 0.6189 - val_loss: 0.5660 - val_accuracy: 0.6150
Epoch 3/10
1960/1960 [==============================] - 160s 81ms/step - loss: 0.5059 - accuracy: 0.6788 - val_loss: 0.5202 - val_accuracy: 0.6473
Epoch 4/10
1960/1960 [==============================] - 158s 81ms/step - loss: 0.4547 - accuracy: 0.7078 - val_loss: 0.4893 - val_accuracy: 0.6710
Epoch 5/10
1960/1960 [==============================] - 161s 82ms/step - loss: 0.4208 - accuracy: 0.7270 - val_loss: 0.4787 - val_accuracy: 0.6881
Epoch 6/10
1960/1960 [==============================] - 163s 83ms/step - loss: 0.3966 - accuracy: 0.7378 - val_loss: 0.4789 - val_accuracy: 0.6916
Epoch 7/10
1960/1960 [==============================] - 163s 83ms/step - loss: 0.3838 - accuracy: 0.7432 - val_loss: 0

NameError: name 'plot_history' is not defined